# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
#print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkifydb 
                        WITH REPLICATION={'class':'SimpleStrategy','replication_factor':'1'}
                        AND durable_writes = TRUE""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.execute("USE sparkifydb")
except Expection as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_music="""CREATE TABLE IF NOT EXISTS music_library
            (sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId,itemInSession,artist))"""
try:
    session.execute(create_music)
except Exception as e:
    print(e)

insert_into_music = "INSERT INTO music_library (sessionId,itemInSession,artist,song,length) VALUES (%s,%s,%s,%s,%s)"

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        data=(int(line[8]),int(line[3]),line[0],line[9],float(line[5])) # sessionId,itemInSession,artist,song,length
        #print("{} {}".format(query,data))
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]
        try:
            session.execute(insert_into_music,data)
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select_music="SELECT * FROM music_library WHERE sessionid =338 and itemInSession=4 "
try:
    results=session.execute(select_music)
    for row in results:
        print(row)
except Exception as e:
    print(e)

Row(sessionid=338, iteminsession=4, artist='Faithless', length=495.30731201171875, song='Music Matters (Mark Knight Dub)')


 sessionid | iteminsession | artist    | length    | song
-----------+---------------+-----------+-----------+---------------------------------
       338 |             4 | Faithless | 495.30731 | Music Matters (Mark Knight Dub)

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
create_artist="""CREATE TABLE IF NOT EXISTS artist_library
             (userId int, sessionId int,artist text, song text, itemInSession int, firstName text, lastName text, PRIMARY KEY ((userId),sessionId,itemInSession))
             WITH CLUSTERING ORDER BY (sessionId ASC, itemInSession ASC);"""

try:
    session.execute(create_artist)
except Exception as e:
    print(e)

insert_artist="INSERT INTO artist_library (artist, song,itemInSession,firstName, lastName,userId, sessionId) VALUES (%s,%s,%s,%s,%s,%s,%s)"                    

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        data=(line[0],line[9],int(line[3]),line[1],line[4],int(line[10]),int(line[8])) 
        #artist,song,itemInSession,firstName,lastName,userId,sessionId,
        #print("{} {}".format(insert_artist,data))
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]
        try:
            session.execute(insert_artist,data)
        except Exception as e:
            print(e)

In [13]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select_music="SELECT * FROM artist_library WHERE userid = 10 and sessionid = 182 "
try:
    results=session.execute(select_music)
    for row in results:
        print(row)
except Exception as e:
    print(e)

Row(userid=10, sessionid=182, iteminsession=0, artist='Down To The Bone', firstname='Sylvie', lastname='Cruz', song="Keep On Keepin' On")
Row(userid=10, sessionid=182, iteminsession=1, artist='Three Drives', firstname='Sylvie', lastname='Cruz', song='Greece 2000')
Row(userid=10, sessionid=182, iteminsession=2, artist='Sebastien Tellier', firstname='Sylvie', lastname='Cruz', song='Kilometer')
Row(userid=10, sessionid=182, iteminsession=3, artist='Lonnie Gordon', firstname='Sylvie', lastname='Cruz', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


In [14]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

create_user="""CREATE TABLE IF NOT EXISTS user_library
            (userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((song),userId,sessionId));"""
try:
    session.execute(create_user)
except Exception as e:
    print(e)

insert_user="INSERT INTO user_library (userId, sessionId, itemInSession, artist, song, firstName, lastName) VALUES (%s,%s,%s,%s,%s,%s,%s)"
                    
                    

In [15]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        data=(int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]) 
        # userId,sessionId,itemInSession,artist,song,firstName,lastName
        #print("{} {}".format(query,data))
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]
        try:
            session.execute(insert_user,data)
        except Exception as e:
            print(e)

In [16]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select_music="SELECT firstName,lastName FROM user_library WHERE song='All Hands Against His Own'"
try:
    results=session.execute(select_music)
    for row in results:
        print(row)
except Exception as e:
    print(e)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [17]:
## TO-DO: Drop the table before closing out the sessions

In [18]:
tables=['music_library','artist_library','user_library']
try:
    for table in tables:
        session.execute("DROP TABLE IF EXISTS {}".format(table))
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()